# Executing workflows

In the previous notebook, we explored how to build custom components, in particular `Controller`s and `Executor`s and have them do some basic logging.

Now, we'll show how to make them interact with each other and execute workflows by implementing their `control_flow` and `execute` method respectively.


## Execution/communication primitives
There are 2 main elements we haven't yet seen that we can use to execute workflows: `Task`s and data-exchange primitives (`Shareable`s and `DXO`s).

- A `Task` represents a unit of work assigned by the controller to the executors. It includes:
    - `name`: The name of the task.
    - `data`: The actual data or instructions associated with the task.
    - Additional properties or metadata.

    How is it used?: The Controller creates tasks and sends them to the clients (we'll dive into how in a bit). Executors receive these tasks and process them in the execute method.

- `Shareable` and `DXO` (Data Exchange Object) are two key data structures used for data-exchange between the server and clients in a federated learning (FL) system. They serve different purposes but are designed to interact closely with each other.

    - A `Shareable` is a flexible data container used for transmitting data between different components in NVFlare. It's essentially a Python dictionary (dict) that can hold any serializable data. It can contain both data and metadata and is used as the primary medium for sending and receiving messages over the network. As said, values must be serializable to ensure they can be transmitted between processes or over the network.

    - A `DXO` provides a standardized way to represent and handle common types of data exchanged during federated learning, such as model weights, gradients, metrics, and hyperparameters. It encapsulates the actual data along with metadata about the data and defines the kind of data being exchanged using `DataKind`. `DXO`s help in maintaining consistency and understanding of data across different components.

## Implementing workflows

In [1]:
from modules import CustomController, CustomExecutor
from nvflare.job_config.api import FedJob

# Create job
job = FedJob(name="dummy_job")

# send controller to server
controller = CustomController()
job.to_server(controller)

# send executor to clients
num_clients = 3
for i in range(num_clients):
    executor = CustomExecutor()
    job.to(executor, f"site-{i}")

# Run job via the NVFlare simulator
job.simulator_run("./tmp/")

2025-02-05 17:58:19,231 - SimulatorRunner - INFO - Create the Simulator Server.
2025-02-05 17:58:19,232 - CoreCell - INFO - server: creating listener on tcp://0:51282
2025-02-05 17:58:19,257 - CoreCell - INFO - server: created backbone external listener for tcp://0:51282
2025-02-05 17:58:19,257 - ConnectorManager - INFO - 48333: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2025-02-05 17:58:19,258 - conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:10645] is starting
2025-02-05 17:58:19,759 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:10645
2025-02-05 17:58:19,760 - conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:51282] is starting
2025-02-05 17:58:19,760 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Request
2025-02-05 17:58:19,760 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Abort
2025-02-05 17:58:19,761 - Cell - INFO - Register blob CB for channel='aux_